In [1]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import sklearn.metrics as sm
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import scipy.misc as sc

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
PATH = '/content/drive/MyDrive/STARE/'
TRAIN_PATH = PATH+'train/'
TRAIN_AUG_PATH = TRAIN_PATH+'aug/'
TRAIN_IMAGE_FOLDER = 'images'
TRAIN_MASK_FOLDER = 'GT'
IMAGE_PREFIX = 'image'
MASK_PREFIX = 'mask'
TRAIN_TARGET_SIZE = (256,256)
TRAIN_BATCH_SIZE = 3
SEED = 1
TEST_PATH = PATH+'test/'
TEST_IMAGE_PATH = TEST_PATH+'images/'
SAVE_TEST_IMAGE_PATH = TEST_PATH+'pred/'
TEST_TARGET_SIZE = (256,256)

In [4]:
def adjustData(img,mask,flag_multi_class,num_class):
    if(flag_multi_class):
        img /= 255
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img) > 1):
        img = img / 255
        mask = mask /255
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    return (img,mask)

def trainGenerator(aug_dict, flag_multi_class = False, num_class = 2):
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        TRAIN_PATH,
        classes = [TRAIN_IMAGE_FOLDER],
        class_mode = None,
        color_mode = 'grayscale',
        target_size = TRAIN_TARGET_SIZE,
        batch_size = TRAIN_BATCH_SIZE,
        save_to_dir = TRAIN_AUG_PATH,
        save_prefix  = IMAGE_PREFIX,
        seed = SEED)
    mask_generator = mask_datagen.flow_from_directory(
        TRAIN_PATH,
        classes = [TRAIN_MASK_FOLDER],
        class_mode = None,
        color_mode = 'grayscale',
        target_size = TRAIN_TARGET_SIZE,
        batch_size = TRAIN_BATCH_SIZE,
        save_to_dir = TRAIN_AUG_PATH,
        save_prefix  = MASK_PREFIX,
        seed = SEED)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)

In [5]:
data_gen_args = dict(rotation_range=0.3,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.1,
                    horizontal_flip=True,
                    fill_mode='nearest')

if not os.path.exists(TRAIN_PATH+'aug'):
    os.makedirs(TRAIN_PATH+'aug')
      
data_gen = trainGenerator(aug_dict=data_gen_args)

https://github.com/iamyb/mobileunet

In [6]:
from keras.layers import Input, SeparableConv2D, BatchNormalization, MaxPooling2D, Conv2DTranspose
from keras.layers import concatenate, Conv2D
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.models import Model
from keras.optimizers import Adam

def mobileunet(pretrained_weights = None,input_size = (256,256,1)):
    inputs = Input(input_size)

    conv1  = SeparableConv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1  = BatchNormalization()(conv1)
    conv1  = SeparableConv2D(64, 3, activation='relu', padding='same')(conv1)
    conv1  = BatchNormalization()(conv1)
    pool1  = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2  = SeparableConv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2  = BatchNormalization()(conv2)
    conv2  = SeparableConv2D(128, 3, activation='relu', padding='same')(conv2)
    conv2  = BatchNormalization()(conv2)
    pool2  = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3  = SeparableConv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3  = BatchNormalization()(conv3)
    conv3  = SeparableConv2D(256, 3, activation='relu', padding='same')(conv3)
    conv3  = BatchNormalization()(conv3)
    pool3  = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4  = SeparableConv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4  = BatchNormalization()(conv4)
    conv4  = SeparableConv2D(512, 3, activation='relu', padding='same')(conv4)
    conv4  = BatchNormalization()(conv4)
    pool4  = MaxPooling2D(pool_size=(2, 2))(conv4)    
    
    conv5  = SeparableConv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5  = BatchNormalization()(conv5)
    conv5  = SeparableConv2D(1024, 3, activation='relu', padding='same')(conv5)
    conv5  = BatchNormalization()(conv5)
    
    conv6  = Conv2DTranspose(512, 3, strides=(2, 2), activation='relu', padding='same')(conv5)
    cat6   = concatenate([conv4, conv6], axis = 3)
    conv6  = SeparableConv2D(512, 3, activation='relu', padding='same')(cat6)
    conv6  = BatchNormalization()(conv6)
    conv6  = SeparableConv2D(512, 3, activation='relu', padding='same')(conv6)
    conv6  = BatchNormalization()(conv6)
    
    conv7  = Conv2DTranspose(256, 3, strides=(2, 2), activation='relu', padding='same')(conv6)
    cat7   = concatenate([conv3, conv7], axis = 3)
    conv7  = SeparableConv2D(256, 3, activation='relu', padding='same')(cat7)
    conv7  = BatchNormalization()(conv7)
    conv7  = SeparableConv2D(256, 3, activation='relu', padding='same')(conv7)
    conv7  = BatchNormalization()(conv7)
    
    conv8  = Conv2DTranspose(128, 3, strides=(2, 2), activation='relu', padding='same')(conv7)
    cat8   = concatenate([conv2, conv8], axis = 3)
    conv8  = SeparableConv2D(128, 3, activation='relu', padding='same')(cat8)
    conv8  = BatchNormalization()(conv8)
    conv8  = SeparableConv2D(128, 3, activation='relu', padding='same')(conv8)    
    conv8  = BatchNormalization()(conv8)
    
    conv9  = Conv2DTranspose(64, 3, strides=(2, 2), activation='relu', padding='same')(conv8)
    cat9   = concatenate([conv1, conv9], axis = 3)
    conv9  = SeparableConv2D(64, 3, activation='relu', padding='same')(cat9)
    conv9  = BatchNormalization()(conv9)
    conv9  = SeparableConv2D(64, 3, activation='relu', padding='same')(conv9)        
    conv9  = BatchNormalization()(conv9)
    conv9  = Conv2D(2, 3, activation='relu', padding='same')(conv9)
    conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)
    
    model = Model(inputs = inputs, outputs = conv10)
    model.compile(optimizer=Adam(lr=1e-3), loss='binary_crossentropy', metrics=['accuracy'])
    
    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

In [7]:
model = mobileunet()
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('unet_STARE.hdf5', monitor='loss', verbose=1, save_best_only=True)
callbacks = [ model_checkpoint ]
history = model.fit(
    data_gen,
    steps_per_epoch=10,
    epochs=80,
    verbose=0,
    callbacks=callbacks)

Found 10 images belonging to 1 classes.
Found 10 images belonging to 1 classes.


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)



Epoch 1: loss improved from inf to 0.56768, saving model to unet_STARE.hdf5

Epoch 2: loss improved from 0.56768 to 0.39824, saving model to unet_STARE.hdf5

Epoch 3: loss improved from 0.39824 to 0.34251, saving model to unet_STARE.hdf5

Epoch 4: loss improved from 0.34251 to 0.32711, saving model to unet_STARE.hdf5

Epoch 5: loss improved from 0.32711 to 0.31625, saving model to unet_STARE.hdf5

Epoch 6: loss improved from 0.31625 to 0.30289, saving model to unet_STARE.hdf5

Epoch 7: loss improved from 0.30289 to 0.29684, saving model to unet_STARE.hdf5

Epoch 8: loss improved from 0.29684 to 0.27390, saving model to unet_STARE.hdf5

Epoch 9: loss improved from 0.27390 to 0.23416, saving model to unet_STARE.hdf5

Epoch 10: loss improved from 0.23416 to 0.21385, saving model to unet_STARE.hdf5

Epoch 11: loss improved from 0.21385 to 0.20035, saving model to unet_STARE.hdf5

Epoch 12: loss improved from 0.20035 to 0.18021, saving model to unet_STARE.hdf5

Epoch 13: loss improved from

[GPU]
start aug: 11.16
train: 15 menit

In [8]:
acc = history.history['accuracy']
loss = history.history['loss']

max_acc = max(acc)
max_acc_i = acc.index(max_acc)
loss_at_max_acc = loss[max_acc_i]

print('acc:', max_acc, 'loss:', loss_at_max_acc)

acc: 0.9777533411979675 loss: 0.0528283417224884


14-02-2023; 23:40 <br>
acc: 0.9706441164016724 loss: 0.09779315441846848 <br><br>

15-02-2023; 01:53 <br>
acc: 0.9781307578086853 loss: 0.06890685111284256 <br>
change: remove zoom scale <br><br>

15-02-2023; 10:16 <br>
acc: 0.9769975543022156 loss: 0.07307402044534683 <br><br>

16-02-2023; 01:44 <br>
chabge: using mobileunet instead of pure unet <br>
acc: 0.9777533411979675 loss: 0.0528283417224884 <br><br>


In [9]:
def testGenerator(as_gray = True):
    files=sorted(os.listdir(TEST_IMAGE_PATH))
    num_image=len(files)
    for i in range(num_image):
        img = io.imread(os.path.join(TEST_IMAGE_PATH,files[i]),as_gray = as_gray)
        print(files[i])
        img = trans.resize(img,TEST_TARGET_SIZE)
        img = np.reshape(img,img.shape+(1,))
        img = np.reshape(img,(1,)+img.shape)
        yield img

In [10]:
def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i] = color_dict[i]
      
    return img_out

def saveResult(npyfile):
    files=os.listdir(TEST_IMAGE_PATH)
    
    for i,item in enumerate(npyfile):
        img = item[:,:,0]
        img[img>0.1]=1
        img[img<=0.1]=0
        io.imsave(os.path.join(SAVE_TEST_IMAGE_PATH, files[i]+'_predict.png'),img)

if not os.path.exists(SAVE_TEST_IMAGE_PATH):
    os.makedirs(SAVE_TEST_IMAGE_PATH)

In [11]:
n_i = len(os.listdir(TEST_IMAGE_PATH))
test_gen = testGenerator()
results = model.predict_generator(test_gen,n_i,verbose=1)
saveResult(results)

<ipython-input-11-046c9d728f71>:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  results = model.predict_generator(test_gen,n_i,verbose=1)


im0162.ppm
10/10 [==============================] - 4s 368ms/step


In [12]:
def get_confusion_matrix_elements(groundtruth_list, predicted_list):
    tn, fp, fn, tp = sm.confusion_matrix(groundtruth_list, predicted_list,labels=[0,1]).ravel()
    tn, fp, fn, tp = np.float64(tn), np.float64(fp), np.float64(fn), np.float64(tp)

    return tn, fp, fn, tp

def get_prec_rec_IoU_accuracy(groundtruth_list, predicted_list):
    tn, fp, fn, tp = get_confusion_matrix_elements(groundtruth_list, predicted_list)
    
    total = tp + fp + fn + tn
    accuracy = (tp + tn) / total
    prec = tp/(tp+fp)
    rec = tp/(tp+fn)
    IoU = tp/(tp+fp+fn)
    
    return prec,rec,IoU,accuracy

def get_f1_score(groundtruth_list, predicted_list):
    tn, fp, fn, tp = get_confusion_matrix_elements(groundtruth_list, predicted_list)
    
    f1_score = (2 * tp) / ((2 * tp) + fp + fn)

    return f1_score

def get_validation_metrics(groundtruth,predicted):   
    u,v = np.shape(groundtruth)
    groundtruth_list = np.reshape(groundtruth,(u*v,))
    predicted_list = np.reshape(predicted,(u*v,))
    prec,rec,IoU,acc = get_prec_rec_IoU_accuracy(groundtruth_list, predicted_list)
    f1_score = get_f1_score(groundtruth_list, predicted_list)

    return prec,rec,IoU,acc,f1_score

def evalResult(gth_path,npyfile,target_size=(512,512),flag_multi_class = False,num_class = 2):
    files=sorted(os.listdir(gth_path))
    print(files)
    prec = 0
    rec = 0
    acc = 0
    IoU = 0
    f1_score=0
    for i,item in enumerate(npyfile):
        img = item[:,:,0]
        gth = io.imread(os.path.join(gth_path,files[i]))
        gth = trans.resize(gth,target_size)
        img1=np.array(((img - np.min(img))/np.ptp(img))>0.1).astype(float)
        gth1=np.array(((gth - np.min(gth))/np.ptp(gth))>0.1).astype(float)
        p,r,I,a,f=get_validation_metrics(gth1,img1)
        prec = prec+p
        rec = rec+r
        acc = acc+a
        IoU = IoU+I
        f1_score = f1_score+f
    print("Precision=",prec/(i+1), "Recall=",rec/(i+1), "IoU=",IoU/(i+1), "acc=",acc/(i+1), "F1=",f1_score/(i+1))    

In [ ]:
gt_path = TEST_PATH + 'GT/'
evalResult(gt_path,results)

### Predicted
Precision= 0.7486418535745212 Recall= 0.8802602326917179 IoU= 0.6784592658474139 acc= 0.9618434906005859 F1= 0.8066515398308723 

<b>15-02-2023; 10:30</b> 
Precision= 0.6528929903566489 Recall= 0.902777067006806 IoU= 0.6070497388289681 acc= 0.9461292266845703 F1= 0.7534104948419812

<b>15-02-2023; 22:33</b> 
Precision= 0.09468648790927088 Recall= 1.0 IoU= 0.09468648790927088 acc= 0.09468994140625 F1= 0.17229373847296356